In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import pandas_data_util
import tensorflow as tf

In [1]:
filename = 'data/bq-results-20211105-102236-1pg8as4q8dgo.csv'
train_percent=0.6
valid_percent=0.2
normalize=0
horizon=0
window=5
Data = pandas_data_util.DataUtil(filename,
                train_percent,
                valid_percent,
                horizon,
                window,
                normalize)

In [ ]:
Data.train[0]